In [1]:
!nvidia-smi

Mon Feb  3 21:49:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 5000 Ada Gene...    Off |   00000000:01:00.0  On |                  Off |
| 30%   31C    P8             22W /  250W |     833MiB /  32760MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Cuda 11
# !pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11

# Cuda 12
# !pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12

In [3]:
import cudf
import pandas as pd

In [4]:
# %load_ext cudf.pandas

In [5]:
df = pd.read_parquet('../data/american_stories_1938_1945.parquet')

In [6]:
df

,article_id,newspaper_name,edition,date,page,headline,byline,article
0,1_1938-11-08_p3_sn82014085_00393347429_1938110...,The Waterbury Democrat.,01,1938-11-08,p3,Fear Heavy Toll Among Civilians In Next Conflict,,Recognition of a probable heavy toll among non...
1,3_1938-11-08_p3_sn82014085_00393347429_1938110...,The Waterbury Democrat.,01,1938-11-08,p3,,,"Conforming to tradition, the Democratic candid..."
2,4_1938-11-08_p3_sn82014085_00393347429_1938110...,The Waterbury Democrat.,01,1938-11-08,p3,Audience Thrilled By\n\n Early Masters Works\n...,,second by Kasper Ferdinand Fisch- CT.\n\n FOlk...
3,5_1938-11-08_p3_sn82014085_00393347429_1938110...,The Waterbury Democrat.,01,1938-11-08,p3,Democrats Institute Court Action T oday\n\n To...,,in behalf of Charles Maloney of Se4 East Main ...
4,6_1938-11-08_p3_sn82014085_00393347429_1938110...,The Waterbury Democrat.,01,1938-11-08,p3,q WOMEN SEEK\n\n ELECTION JOBS\n\nNone Candida...,BY RUBY A. BLACK nifAd preea Sfaff Corresnovad...,NUnlte0 freSS Stam COrreSpOn0ent)\n\n Washingt...
...,...,...,...,...,...,...,...,...
4368783,28_1945-12-05_p7_sn88063294_00340589130_194512...,Detroit evening times.,01,1945-12-05,p7,,,Here's a contest you win! And what prize!\nA b...
4368784,10_1945-04-18_p35_sn83045462_00280604082_19450...,Evening star.,01,1945-04-18,p35,"ADVERTISEMENT,\n\nADNL n lSLhLt>1\nTorment OF ...",,II you can't get your feet of your mind\nbecau...
4368785,3_1945-04-18_p35_sn83045462_00280604082_194504...,Evening star.,01,1945-04-18,p35,Jury Rules Chaplin\nIs.Father;; Conference\nOn...,By the Associated Press.,"By the Associated Press.\n\n\nLOS ANGELES, Apr..."
4368786,21_1945-04-18_p35_sn83045462_00280604082_19450...,Evening star.,01,1945-04-18,p35,Use Your\nBeIdqet Account,,"It's easy as A-B-C to open a\nCharge, Budget o..."


In [7]:
def find_patterns(pattern, df, column='article'):
    return df[df[column].str.contains(pattern, regex=True)].copy()

In [8]:
%%time
# Search for capitalized first and last names (e.g., "John Smith")
# Find all capitalized words in articles
name_pattern = r'[A-Z][a-z]+'
column = 'article'
names_search = find_patterns(name_pattern, df, column)

# Extract all capitalized words from each article
names_search['capitalized_words'] = names_search[column].str.findall(name_pattern)

print("Number of articles with capitalized words:", len(names_search))
names_search[['article_id', 'article', 'capitalized_words']].head()


Number of articles with capitalized words: 4329309
CPU times: user 33.3 s, sys: 1.79 s, total: 35.1 s
Wall time: 35.1 s


,article_id,article,capitalized_words
0,1_1938-11-08_p3_sn82014085_00393347429_1938110...,Recognition of a probable heavy toll among non...,"[Recognition, International, Conference, Paris..."
1,3_1938-11-08_p3_sn82014085_00393347429_1938110...,"Conforming to tradition, the Democratic candid...","[Conforming, Democratic, Dnnell, Bridge, Attor..."
2,4_1938-11-08_p3_sn82014085_00393347429_1938110...,second by Kasper Ferdinand Fisch- CT.\n\n FOlk...,"[Kasper, Ferdinand, Fisch, Olk, Ong, Next, Won..."
3,5_1938-11-08_p3_sn82014085_00393347429_1938110...,in behalf of Charles Maloney of Se4 East Main ...,"[Charles, Maloney, Se, East, Main, Attorney, L..."
4,6_1938-11-08_p3_sn82014085_00393347429_1938110...,NUnlte0 freSS Stam COrreSpOn0ent)\n\n Washingt...,"[Unlte, Stam, Orre, Sp, On, Washington, Nov, T..."


In [9]:
cuda_df = cudf.read_parquet("../data/american_stories_1938_1945.parquet")

In [10]:
%%time


names_search_cudf = find_patterns(name_pattern, cuda_df, column='article')

# Extract all capitalized words from each article
names_search_cudf['capitalized_words'] = names_search_cudf['article'].str.findall(name_pattern)

# Display sample of matches with capitalized words
print("Number of articles with capitalized words:", len(names_search_cudf))
names_search[['article_id', 'article', 'capitalized_words']].head()

Number of articles with capitalized words: 4329309
CPU times: user 2.72 s, sys: 95.9 ms, total: 2.81 s
Wall time: 2.81 s


,article_id,article,capitalized_words
0,1_1938-11-08_p3_sn82014085_00393347429_1938110...,Recognition of a probable heavy toll among non...,"[Recognition, International, Conference, Paris..."
1,3_1938-11-08_p3_sn82014085_00393347429_1938110...,"Conforming to tradition, the Democratic candid...","[Conforming, Democratic, Dnnell, Bridge, Attor..."
2,4_1938-11-08_p3_sn82014085_00393347429_1938110...,second by Kasper Ferdinand Fisch- CT.\n\n FOlk...,"[Kasper, Ferdinand, Fisch, Olk, Ong, Next, Won..."
3,5_1938-11-08_p3_sn82014085_00393347429_1938110...,in behalf of Charles Maloney of Se4 East Main ...,"[Charles, Maloney, Se, East, Main, Attorney, L..."
4,6_1938-11-08_p3_sn82014085_00393347429_1938110...,NUnlte0 freSS Stam COrreSpOn0ent)\n\n Washingt...,"[Unlte, Stam, Orre, Sp, On, Washington, Nov, T..."
